In [1]:
import numpy as np
import glob
import os
import lsst.afw.image as afwImage

from lsst.summit.utils.astrometry.utils import runCharactierizeImage, genericCameraHeaderToWcs
from lsst.summit.utils.astrometry.anet import CommandLineSolver

In [2]:
def solveExposure(exp, wideField):
    solver = CommandLineSolver(indexFilePath='/scratch/mfisherlevine/astrometry_net/broiler.astrometry.net/~dstn/')
    
    try:
        wcs = genericCameraHeaderToWcs(exp)
    except RuntimeError:
        return False

    exp.setWcs(wcs)
    results = runCharactierizeImage(exp, 5, 25)
        
    if wideField == True:
        newWcs = solver.run(exp, results.sourceCat, True)
    else:
        newWcs = solver.run(exp, results.sourceCat, True, percentageScaleError=5)

    if newWcs != None:           
        exp.setWcs(newWcs.wcs)
        return True
    else:
        return False

In [3]:
imageDir = '/home/adriansh95/DATA/starTracker/tmaTests/procFits/' # Adrian summit
solvedDir = '/home/adriansh95/DATA/starTracker/tmaTests/solvedFits/'
cam101 = 'GC101'
cam102 = 'GC102'
date = '20221207'
stems = ''
globStr101 = os.path.join(imageDir, f'{cam101}*{date}*{stems}*.fits')
globStr102 = os.path.join(imageDir, f'{cam102}*{date}*{stems}*.fits')
files101 = sorted(glob.glob(globStr101))
files102 = sorted(glob.glob(globStr102))

In [7]:
# Use the first image to "calibrate" i.e. map the boresight pixel from the wide Cam
# onto a pixel in the narrow cam by doing: boresightPixel101 -> boresightSky101 (skyPoint) -> boresightPixel102
calibrationExpStr = '20221207_000004'
calibrationExpFile101 = [f for f in files101 if calibrationExpStr in f][0]
calibrationExpFile102 = [f for f in files102 if calibrationExpStr in f][0]
print(calibrationExpFile101, calibrationExpFile102)
boresightPixel101 = np.array([1257.151, 1560.85])
calibrationExp101, calibrationExp102 = afwImage.ExposureF(calibrationExpFile101), afwImage.ExposureF(calibrationExpFile102)
solved101, solved102 = solveExposure(calibrationExp101, True), solveExposure(calibrationExp102, False)
boresightSky101 = calibrationExp101.getWcs().pixelToSky(boresightPixel101[1], boresightPixel101[0])
boresightPixel102 = calibrationExp102.getWcs().skyToPixel(boresightSky101)
boresightPixel101 = calibrationExp101.getWcs().skyToPixel(boresightSky101) # turn boresightPixel101 into a Point2D object

/home/adriansh95/DATA/starTracker/tmaTests/procFits/GC101_O_20221207_000004.fits /home/adriansh95/DATA/starTracker/tmaTests/procFits/GC102_O_20221207_000004.fits


Fitting image with 432 sources of which 432 made it into the fit
Fitting code ran in 2.37 seconds


Fitting image with 265 sources of which 265 made it into the fit
Fitting code ran in 2.42 seconds


In [8]:
boresightSky102 = calibrationExp102.getWcs().pixelToSky(boresightPixel102)
print(boresightSky101.separation(boresightSky102)) # Just verifying that the two points line up exactly on sky

8.47633e-11 rad


In [9]:
# Solve a bunch of starTracker images and record the (scalar) angular separation on sky of boresightPixel101 and boresightPixel102
separations = []

for f1, f2 in zip(files101[3:], files102[3:]):
    print(f1, f2)
    exp1, exp2 = afwImage.ExposureF(f1), afwImage.ExposureF(f2)
    solved1, solved2 = solveExposure(exp1, True), solveExposure(exp2, False)
    
    # Continue onto next pair of images if one or more images don't solve
    if not (solved1 and solved2):
        continue
            
    baseName1, baseName2 = os.path.basename(f1).split('.')[0], os.path.basename(f2).split('.')[0]
    fullPath1, fullPath2 = os.path.join(solvedDir, baseName1), os.path.join(solvedDir, baseName2)
    writeFile1, writeFile2 = f"{fullPath1}_solved.fits", f"{fullPath2}_solved.fits"
    exp1.writeFits(writeFile1)
    exp2.writeFits(writeFile2)
    print(f"Wrote {writeFile1}", f"Wrote {writeFile2}")
    
    # Map the boresightPixels onto sky
    skyPoint1 = exp1.getWcs().pixelToSky(boresightPixel101)
    skyPoint2 = exp2.getWcs().pixelToSky(boresightPixel102)
    
    # Compute angular separation
    angularOffset = skyPoint1.separation(skyPoint2).asArcseconds()
    separations.append(angularOffset)

/home/adriansh95/DATA/starTracker/tmaTests/procFits/GC101_O_20221207_000004.fits /home/adriansh95/DATA/starTracker/tmaTests/procFits/GC102_O_20221207_000004.fits


Fitting image with 432 sources of which 432 made it into the fit
Fitting code ran in 2.41 seconds


Fitting image with 265 sources of which 265 made it into the fit
Fitting code ran in 2.42 seconds
Wrote /home/adriansh95/DATA/starTracker/tmaTests/solvedFits/GC101_O_20221207_000004_solved.fits Wrote /home/adriansh95/DATA/starTracker/tmaTests/solvedFits/GC102_O_20221207_000004_solved.fits
/home/adriansh95/DATA/starTracker/tmaTests/procFits/GC101_O_20221207_000005.fits /home/adriansh95/DATA/starTracker/tmaTests/procFits/GC102_O_20221207_000005.fits


Fitting image with 437 sources of which 437 made it into the fit
Fitting code ran in 2.42 seconds


Fitting image with 255 sources of which 255 made it into the fit
Fitting code ran in 2.31 seconds
Wrote /home/adriansh95/DATA/starTracker/tmaTests/solvedFits/GC101_O_20221207_000005_solved.fits Wrote /home/adriansh95/DATA/starTracker/tmaTests/solvedFits/GC102_O_20221207_000005_solved.fits
/home/adriansh95/DATA/starTracker/tmaTests/procFits/GC101_O_20221207_000006.fits /home/adriansh95/DATA/starTracker/tmaTests/procFits/GC102_O_20221207_000006.fits


...

Fitting image with 147 sources of which 147 made it into the fit
Fitting code ran in 2.57 seconds
Wrote /home/adriansh95/DATA/starTracker/tmaTests/solvedFits/GC101_O_20221207_000042_solved.fits Wrote /home/adriansh95/DATA/starTracker/tmaTests/solvedFits/GC102_O_20221207_000042_solved.fits
/home/adriansh95/DATA/starTracker/tmaTests/procFits/GC101_O_20221207_000043.fits /home/adriansh95/DATA/starTracker/tmaTests/procFits/GC102_O_20221207_000043.fits


Fitting image with 271 sources of which 271 made it into the fit
Fitting code ran in 2.40 seconds


Fitting image with 71 sources of which 71 made it into the fit
Fitting code ran in 2.37 seconds
Wrote /home/adriansh95/DATA/starTracker/tmaTests/solvedFits/GC101_O_20221207_000043_solved.fits Wrote /home/adriansh95/DATA/starTracker/tmaTests/solvedFits/GC102_O_20221207_000043_solved.fits
/home/adriansh95/DATA/starTracker/tmaTests/procFits/GC101_O_20221207_000044.fits /home/adriansh95/DATA/starTracker/tmaTests/procFits/GC102_O_20221207_000044.fits


Fitting image with 4 sources of which 4 made it into the fit
Fitting code ran in 2.26 seconds
but failed to find a solution.


Fitting image with 69 sources of which 69 made it into the fit
Fitting code ran in 2.15 seconds
